# 10 minutes to gators

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import copy
import numpy as np
import pandas as pd
from pandas.testing import assert_frame_equal
from xgboost import XGBClassifier
import treelite
import treelite_runtime
import dill

/Users/cpoli/gators38/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [4]:
# imputers
from gators.imputers import (
    NumericsImputer, 
    ObjectImputer,
)
# encoders
from gators.encoders import WOEEncoder
# pipeline
from gators.pipeline import Pipeline
# model building
from gators.model_building import XGBBoosterBuilder

## end-to-end simple worflow

The workflow is as followed:

1- create a pipeline to take care of the data preproprocessing.

2- train the pipeline on a *pandas* or *koalas* dataframe.

3- generate the preproccessed data.

4- train a decision tree based model on the preprocessed data.

5- use *treelite* to compile the model in C.
    
The pipeline and the compiled model can then be deployed in production.

**Notes:**
    
* *koalas* and/or *pandas* are used offline,
by means of the `fit` and `transform methods`.
* In production, *numpy* is used with `transform_numpy`.

The pipeline will be only composed of the following four transformers:

* ObjectImputer
* WOEEncoder
* NumericsImputer

### with pandas

In [5]:
columns = ['Sex', 'Age', 'SibSp', 'Parch', 'Cabin', 'Survived']
data = pd.read_parquet('data/titanic.parquet')[columns]
y = data['Survived']
X = data.drop(['Survived'], axis=1)
X.head(2)

,Sex,Age,SibSp,Parch,Cabin
0,male,22.0,1,0,None
1,female,38.0,1,0,C85


#### pipeline

In [6]:
prepro_steps = [
    ('ObjectImputer', ObjectImputer(strategy='constant', value='MISSING')),
    ('WOEEncoder', WOEEncoder()),
    ('NumericsImputer', NumericsImputer(strategy='mean')),
]
pipe = Pipeline(steps=prepro_steps)
X_prepro = pipe.fit_transform(X, y)
X_prepro_np = pipe.transform_numpy(X.to_numpy())

In [7]:
X_prepro.head(2)

,Sex,Age,SibSp,Parch,Cabin
0,-1.453612,22.0,1.0,0.0,-0.846606
1,1.052579,38.0,1.0,0.0,1.098612


##### check `transform` and `tranform_numpy` output

In [8]:
assert X_prepro.shape == X_prepro_np.shape
X_prepro_np_pd = pd.DataFrame(
    X_prepro_np, 
    index=X_prepro.index, 
    columns=X_prepro.columns,
)
assert_frame_equal(X_prepro, X_prepro_np_pd)

### model building

In [9]:
model = XGBClassifier(
    max_depth=2,
    n_estimators=10,
    random_state=0, 
    eval_metric='mlogloss', 
    use_label_encoder=False)

#### pandas model

In [10]:
model_pd = copy.copy(model)
_ = model_pd.fit(X_prepro, y)

/Users/cpoli/gators38/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


#### numpy model

In [11]:
model = model.fit(X_prepro.to_numpy(), y.to_numpy())

#### treelite model

In [12]:
xgb_booster = XGBBoosterBuilder.train(
    model=model, 
    X_train=X_prepro_np, 
    y_train=y.to_numpy(),
)

treelite_model = treelite.Model.from_xgboost(xgb_booster)
treelite_model.export_lib(
    toolchain='gcc', 
    libpath='./models/treelite_simple_xgb.so', 
    params={'parallel_comp': 4},
    verbose=True)
model_tl = treelite_runtime.Predictor(
    './models/treelite_simple_xgb.so', verbose=False)

[07:01:29] /Users/travis/build/dmlc/treelite/src/compiler/ast_native.cc:711: Using ASTNativeCompiler
[07:01:29] /Users/travis/build/dmlc/treelite/src/compiler/ast/split.cc:29: Parallel compilation enabled; member trees will be divided into 4 translation units.
[07:01:29] /Users/travis/build/dmlc/treelite/src/c_api/c_api.cc:92: Code generation finished. Writing code to files...
[07:01:29] /Users/travis/build/dmlc/treelite/src/c_api/c_api.cc:97: Writing file recipe.json...
[07:01:29] /Users/travis/build/dmlc/treelite/src/c_api/c_api.cc:97: Writing file tu3.c...
[07:01:29] /Users/travis/build/dmlc/treelite/src/c_api/c_api.cc:97: Writing file tu2.c...
[07:01:29] /Users/travis/build/dmlc/treelite/src/c_api/c_api.cc:97: Writing file tu1.c...
[07:01:29] /Users/travis/build/dmlc/treelite/src/c_api/c_api.cc:97: Writing file tu0.c...
[07:01:29] /Users/travis/build/dmlc/treelite/src/c_api/c_api.cc:97: Writing file header.h...
[07:01:29] /Users/travis/build/dmlc/treelite/src/c_api/c_api.cc:97: Wri

##### per-sample model benchmarking

In [13]:
x = X.iloc[[0]]
xnp = x.to_numpy()
stats_pd = %timeit -o model_pd.predict_proba(pipe.transform(x))[0][1]
stats_pd_tl = %timeit -o model_tl.predict(treelite_runtime.DMatrix(pipe.transform(x).to_numpy()))
stats_np = %timeit -o model.predict_proba(pipe.transform_numpy(xnp))[0][1]
stats_np_tl = %timeit -o model_tl.predict(treelite_runtime.DMatrix(pipe.transform_numpy(xnp)))

10.3 ms ± 1.07 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
6.22 ms ± 374 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
1.28 ms ± 50.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
53.8 µs ± 2.69 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


Overall speed-up

In [14]:
speedup = 1e3 * float(str(stats_pd).split(' ')[0]) / float(str(stats_np_tl).split(' ')[0])
f'Speed-up Pandas VS Numpy&Treelite x{round(speedup)}'

'Speed-up Pandas VS Numpy&Treelite x191'

#### check model predictions

In [15]:
X_np = X.to_numpy()
y_pred_pd = model_pd.predict_proba(pipe.transform(X))[:, 1]
y_pred_np = model.predict_proba(pipe.transform_numpy(X_np))[:, 1]
y_pred_tl = model_tl.predict(treelite_runtime.DMatrix(pipe.transform_numpy(X_np).astype(float)))
assert np.allclose(y_pred_np, y_pred_pd)
assert np.allclose(y_pred_np, y_pred_tl)

#### dumping both model and pipeline 

In [16]:
model_path = 'models/simple_xgb.dill'
with open(model_path, 'wb') as file:
    dill.dump(model, file)
model_path = 'models/simple_pipeline.dill'
with open(model_path, 'wb') as file:
    dill.dump(model, file)

### with koalas

In [17]:
from pyspark import SparkConf, SparkContext

conf = SparkConf()
conf.set('spark.executor.memory', '2g')
conf.set('spark.sql.codegen.wholeStage', 'false')
conf.set('spark.sql.autoBroadcastJoinThreshold', -1)
SparkContext(conf=conf)
import databricks.koalas as ks
ks.set_option('compute.default_index_type', 'distributed-sequence')

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/02 07:02:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [18]:
import databricks.koalas as ks
from gators.converter import ToPandas

In [19]:
data = ks.read_parquet('data/titanic.parquet')[columns]
y_ks = data['Survived']
X_ks = data.drop(['Survived', 'PassengerId'], axis=1)
X_ks.head(2)

,Sex,Age,SibSp,Parch,Cabin
0,male,22.0,1,0,None
1,female,38.0,1,0,C85


In [20]:
X_prepro_ks = pipe.fit_transform(X_ks, y_ks)
X_prepro_ks_np = pipe.transform_numpy(X.to_numpy())

In [21]:
X_prepro_ks_pd, y_ks_pd = ToPandas().transform(X_prepro_ks, y_ks)
X_prepro_ks_pd.index = X_prepro.index

##### check `pandas` and `koalas` output

In [22]:
assert_frame_equal(X_prepro_ks_pd, X_prepro)
assert np.allclose(X_prepro_ks_np, X_prepro)

Since we have a pandas dataframe, the same steps from the pandas section can now followed

## Create you own transformers: example with log10

### imports

In [23]:
from typing import List, Union
from math import log10
import numpy as np
import pandas as pd
import databricks.koalas as ks
from gators.util import util
from gators.transformers import Transformer

### Inplace transformer on the all dataframe

In [24]:
class Log10Inplace(Transformer):
    def __init__(self):
        pass

    def fit(self,
            X: Union[pd.DataFrame, ks.DataFrame],
            y: Union[pd.Series, ks.Series] = None) -> 'Log10Inplace':
        self.check_dataframe(X)
        return self

    def transform(
        self, X: Union[pd.DataFrame, ks.DataFrame]
    ) -> Union[pd.DataFrame, ks.DataFrame]:
        self.check_dataframe(X)
        return X.applymap(log10)

    def transform_numpy(self, X: np.ndarray) -> np.ndarray:
        self.check_array(X)
        return np.log10(X)

**Notes:**

If your use case do not need koalas, the `transform` method can be replaced by:
 
`return pd.DataFrame(np.log10(X.to_numpy()), columns=X.columns, index=X.index)`

which is significantly faster.

In [25]:
X = pd.DataFrame(
    np.abs(np.random.randn(10, 10)), columns=list('ABCDEFGHIJ'))
%timeit _ = X.applymap(log10)
%timeit _ = pd.DataFrame(np.log10(X.to_numpy()), columns=X.columns, index=X.index)

1.43 ms ± 18.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
31.3 µs ± 1 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Inplace transformer on the selected columns

Transform the given columns can be interesting in the following ways:

* Only a few colums need to be transformed.
* Only a given datatype should be transformed.
* The transformation should not be applied on the encoded columns, and the name of the base columns are obtained before the transformation.

In [26]:
class Log10ColumnsInplace(Transformer):
    def __init__(self, columns: List[str]):
        if not isinstance(columns, list):
            raise TypeError('`columns` should be a list.')
        if not columns:
            raise ValueError('`columns` should not be empty.')
        self.columns = columns
        
    def fit(self,
            X: Union[pd.DataFrame, ks.DataFrame],
            y: Union[pd.Series, ks.Series] = None) -> 'Log10Columns':
        self.check_dataframe(X)
        self.idx_columns = util.get_idx_columns(
            columns=X.columns,
            selected_columns=self.columns
        )
        return self

    def transform(
        self, X: Union[pd.DataFrame, ks.DataFrame]
    ) -> Union[pd.DataFrame, ks.DataFrame]:
        self.check_dataframe(X)
        X[self.columns] = X[self.columns].applymap(log10)
        return X

    def transform_numpy(self, X: np.ndarray) -> np.ndarray:
        self.check_array(X)
        X[:, self.idx_columns] = np.log10(X[:, self.idx_columns])
        return X
    

### transformer creating new columns

Creating new columns can be interesting if

* the raw data are needed for other transformations. 
* the raw data still contains some meaningful predictive information. 

In [27]:
class Log10Columns(Transformer):
    def __init__(self, columns: List[str]):
        if not isinstance(columns, list):
            raise TypeError('`columns` should be a list.')
        if not columns:
            raise ValueError('`columns` should not be empty.')
        self.columns = columns
        self.column_names = [f'{c}__log10' for c in self.columns]

    def fit(self,
            X: Union[pd.DataFrame, ks.DataFrame],
            y: Union[pd.Series, ks.Series] = None) -> 'Log10Columns':
        self.check_dataframe(X)
        self.idx_columns = util.get_idx_columns(
            columns=X.columns,
            selected_columns=self.columns
        )
        return self

    def transform(
        self, X: Union[pd.DataFrame, ks.DataFrame]
    ) -> Union[pd.DataFrame, ks.DataFrame]:
        self.check_dataframe(X)
        X_new = X[self.columns].applymap(log10)
        X_new.columns = self.column_names
        return X.join(X_new)

    def transform_numpy(self, X: np.ndarray) -> np.ndarray:
        self.check_array(X)
        X_new = np.log10(X[:, self.idx_columns])
        return np.concatenate((X, X_new), axis=1)

### tests

In [28]:
X = pd.DataFrame(
    np.abs(np.random.randn(10, 10)), columns=list('ABCDEFGHIJ'))
X_np = X.to_numpy()

In [29]:
columns = util.get_datatype_columns(X, float)
X_new_inplace_all = Log10Inplace().fit_transform(X.copy())

In [30]:
columns = util.get_datatype_columns(X, float)
X_new_inplace_cols = Log10ColumnsInplace(
    columns=columns).fit_transform(X.copy())

In [31]:
columns = util.get_datatype_columns(X, float)
X_new = Log10Columns(columns=columns).fit_transform(X)

In [32]:
assert np.allclose(
    X_new_inplace_all.to_numpy(), X_new_inplace_cols.to_numpy())
cols = [
    'A__log10', 'B__log10', 'C__log10', 'D__log10', 'E__log10', 
    'F__log10', 'G__log10', 'H__log10', 'I__log10', 'J__log10'
]
assert np.allclose(
    X_new_inplace_all.to_numpy(), X_new[cols].to_numpy())

### per-sample benchmarking

In [33]:
np.random.seed(0)
n_cols = 1000
X = pd.DataFrame(
    np.abs(np.random.randn(1, n_cols)), 
    columns=[f'col{i}'for i in range(n_cols)])
X_np = X.to_numpy()
x = X.iloc[[0]]
x_np = x.to_numpy()

In [34]:
columns = list(X.columns)
obj = Log10ColumnsInplace(columns=columns)
_ = obj.fit(X)

In [35]:
# result saved to be compared with the transform_numpy using Cython.
x_np_new = obj.transform_numpy(x_np.copy())

In [36]:
%timeit _ = obj.transform(x.copy())
%timeit _ = obj.transform_numpy(x_np.copy())

237 ms ± 3.98 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
14.8 µs ± 453 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


**Notes:**
    
Since the transformation happens inplace, the `.copy()` is 
neccessary however, the `.copy()` runtime is negligeable: 

In [37]:
%timeit x.copy()
%timeit x_np.copy()

18 µs ± 440 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
585 ns ± 22.7 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [38]:
columns = list(X.columns)
obj = Log10Columns(columns=columns)
_ = obj.fit(X)

In [39]:
%timeit _ = obj.transform(x.copy())
%timeit _ = obj.transform_numpy(x_np.copy())

99.4 ms ± 2.16 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
14.9 µs ± 508 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


### Cython

The per-sample runtime of the `transform_numpy` is already pretty good.
But, it some cases, Cython will allow to get even faster.  

In [40]:
%load_ext Cython

In [41]:
%%cython
import cython
import numpy as np
cimport numpy as np
from libc.math cimport log10


@cython.boundscheck(False)
@cython.wraparound(False)
cpdef np.ndarray[np.float64_t, ndim=2] cython_log10(
        np.ndarray[np.float_t, ndim=2] X,
        np.ndarray[np.int64_t, ndim=1] idx_columns,
):
    cdef int i
    cdef int j
    cdef int n_rows = X.shape[0]
    cdef int n_cols = X.shape[1]
    with nogil:
        for i in range(n_rows):
            for j in range(n_cols):
                X[i, j] = log10(X[i, j])
    return X

In [42]:
class Log10ColumnsInplaceWithCython(Transformer):
    def __init__(self, columns: List[str]):
        if not isinstance(columns, list):
            raise TypeError('`columns` should be a list.')
        if not columns:
            raise ValueError('`columns` should not be empty.')
        self.columns = columns
        
    def fit(self, X, y=None) -> 'Log10Columns':
        self.check_dataframe(X)
        self.idx_columns = util.get_idx_columns(
            columns=X.columns,
            selected_columns=self.columns
        )
        return self

    def transform(self, X):
        self.check_dataframe(X)
        X[self.columns] = X[self.columns].applymap(log10)
        return X

    def transform_numpy(self, X):
        self.check_array(X)
        X[:, self.idx_columns] = cython_log10(X, self.idx_columns)
        return X

In [43]:
columns = list(X.columns)
obj = Log10ColumnsInplaceWithCython(columns=columns)
_ = obj.fit(X)

In [44]:
assert np.allclose(obj.transform_numpy(x_np.copy()), x_np_new)

In [45]:
%timeit _ = obj.transform_numpy(x_np.copy())

10.9 µs ± 494 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


A slight runtime improvement is obtained for this transformer.

**Notes:**

In some cases, for example the Encoders, Cython leads to a significant runtime improvement. 